# Test Rubric for the Final Lab Project



## Initial setup. 

Do not change anything from next cell

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

print(torch.__version__)
print(torch.cuda.is_available())

!pip install --no-cache-dir -I pillow

0.4.1
True
    100% |████████████████████████████████| 2.0MB 27.6MB/s 


## Loading your checkpoint file

In [0]:
!pip install -U -q PyDrive

import tensorflow as tf
import timeit

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

from google.colab import drive
drive.mount('/content/drive')

0.4.1
True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Your submission

In [0]:
from collections import OrderedDict

import torch
import torch.nn as nn
from torchvision import models

def load_checkpoint(checkpoint_path):
    # define your model and return it
    model = models.resnet152(pretrained=True) # densenet121
    # Turning off gradiants features
    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(2048, 1024)),
                                ('relu1', nn.ReLU()),
                                ('dropout1', nn.Dropout(p=0.5)),
                                ('fc2', nn.Linear(1024, 512)),
                                ('relu2', nn.ReLU()),
                                ('dropout1', nn.Dropout(p=0.3)),
                                ('fc3', nn.Linear(512, 102))
                                ]))

    model.load_state_dict(torch.load(checkpoint_path))
    model.to("cuda")
    
    return model

# Load your model to this variable
model = load_checkpoint('drive/My Drive/AI/flower_model.pt')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

## Evaluation

In [0]:
# we will download the required data files
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

rm: cannot remove 'flower_data': No such file or directory


In [0]:
# required imports
import os

from torchvision import datasets, transforms


In [0]:
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)
    ])

data_dir = 'drive/My Drive/AI/flower_data/'
x = 'valid'
image_dataset = datasets.ImageFolder(os.path.join(data_dir, x), data_transforms)
dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=32, shuffle=True, num_workers=4)
print(len(dataloader))

29


In [0]:
def calculate_accuracy(model, dataloader):
    model.eval()
    with torch.no_grad():
        total_predictions_vector = None
        for index, (inputs, labels) in enumerate(dataloader):
            inputs.to(torch.device("cpu"))
            labels.to(torch.device("cpu"))
            outputs = model.forward(inputs)
            _, y_hat = outputs.max(1)
            current_batch_predictions_vector = (y_hat==labels.data)
            if index == 0:
                total_predictions_vector = current_batch_predictions_vector
            else:
              total_predictions_vector = torch.cat(( current_batch_predictions_vector, total_predictions_vector), 0)

        total_predictions_vector = total_predictions_vector.cpu()
        valid_items = total_predictions_vector.sum().numpy()
        total_items = list(total_predictions_vector.size())[0]
        accuracy = valid_items/total_items
        return accuracy

In [0]:

print("Accuracy on validation set you have got is: ", calculate_accuracy(model, dataloader))

In [0]:
!pip install -q pillow==4.1.1
!git clone https://github.com/GabrielePicco/deep-learning-flower-identifier
!pip install requests
!pip install airtable
import sys
sys.path.insert(0, 'deep-learning-flower-identifier')
from test_model_pytorch_facebook_challenge import calc_accuracy

calc_accuracy(model, input_image_size=224)

fatal: destination path 'deep-learning-flower-identifier' already exists and is not an empty directory.
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.875
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.875
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.90625
Batch accuracy (Size 32): 0.9375
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Mean accuracy: 0.9495192170143127


0.9495192